In [1]:
!apt-get install git -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
!git clone https://github.com/kingbedjed/2025_Hackathon


Cloning into '2025_Hackathon'...
remote: Enumerating objects: 2259, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 2259 (delta 54), reused 162 (delta 43), pack-reused 2083 (from 2)
Receiving objects: 100% (2259/2259), 77.36 MiB | 11.83 MiB/s, done.
Resolving deltas: 100% (525/525), done.


In [3]:
!cp /content/drive/MyDrive/Colab Notebooks/vlm_defect_classification.ipynb /content/2025_Hackathon


cp: cannot stat '/content/drive/MyDrive/Colab': No such file or directory
cp: cannot stat 'Notebooks/vlm_defect_classification.ipynb': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

BASE_DIR = '/content/drive/MyDrive/hackathon_2025'
os.makedirs(BASE_DIR, exist_ok=True)
BASE_DIR


'/content/drive/MyDrive/hackathon_2025'

In [ ]:
import os
import shutil
from glob import glob

RAW_DIR = '/content/drive/MyDrive/hackathon_2025/DataSet_CdTe'
OUT_DIR_TIF = '/content/drive/MyDrive/hackathon_2025/defect_patches'
os.makedirs(OUT_DIR_TIF, exist_ok=True)

images = sorted(glob(os.path.join(RAW_DIR, '*')))

for i, path in enumerate(images):
    ext = os.path.splitext(path)[-1].lower()
    new_name = f"{i:05d}{ext}"  # 00000.png → 01118.png
    shutil.copy(path, os.path.join(OUT_DIR_TIF, new_name))

len(images)


1119

In [ ]:
CONVERTED_DIR = '/content/drive/MyDrive/hackathon_2025/DataSet_CdTe_png'
os.makedirs(CONVERTED_DIR, exist_ok=True)

tif_files = sorted(glob(os.path.join(RAW_DIR, "*.tif")))

print("Found", len(tif_files), "TIFF files")

for tif_path in tif_files:
    img = Image.open(tif_path)
    img = img.convert("L")  # grayscale
    new_name = os.path.splitext(os.path.basename(tif_path))[0] + ".png"
    out_path = os.path.join(CONVERTED_DIR, new_name)
    img.save(out_path, format="PNG")

print("Conversion complete:", len(tif_files), "files converted.")

Found 1119 TIFF files
Conversion complete: 1119 files converted.


In [ ]:
OUT_DIR = '/content/drive/MyDrive/hackathon_2025/defect_patches_png'
os.makedirs(OUT_DIR, exist_ok=True)

png_files = sorted(glob(os.path.join(CONVERTED_DIR, "*.png")))
print("Found", len(png_files), "PNG files to rename.")

for idx, path in enumerate(png_files):
    new_name = f"{idx:05d}.png"   # 00000.png, 00001.png, ...
    shutil.copy(path, os.path.join(OUT_DIR, new_name))

print("Renaming complete:", len(png_files), "files saved to", OUT_DIR)


Found 1119 PNG files to rename.
Renaming complete: 1119 files saved to /content/drive/MyDrive/hackathon_2025/defect_patches_png


In [ ]:
import pandas as pd

manifest = []

for idx, path in enumerate(png_files):
    original = os.path.basename(path)
    new_name = f"{idx:05d}.png"
    manifest.append({"new_name": new_name, "original_name": original})

df = pd.DataFrame(manifest)
df.to_csv(os.path.join(BASE_DIR, "manifest_png.csv"), index=False)

df.head()


,new_name,original_name
0,00000.png,Bulk_0.png
1,00001.png,Bulk_1.png
2,00002.png,Bulk_10.png
3,00003.png,Bulk_100.png
4,00004.png,Bulk_101.png


In [ ]:
import pandas as pd

manifest = []

for i, path in enumerate(images):
    ext = os.path.splitext(path)[-1].lower()
    new_name = f"{i:05d}{ext}"
    manifest.append({
        'new_name': new_name,
        'original_name': os.path.basename(path)
    })

df = pd.DataFrame(manifest)
df.to_csv(os.path.join(BASE_DIR, 'manifest.csv'), index=False)
df.head()


,new_name,original_name
0,00000.tif,Bulk_0.tif
1,00001.tif,Bulk_1.tif
2,00002.tif,Bulk_10.tif
3,00003.tif,Bulk_100.tif
4,00004.tif,Bulk_101.tif


In [ ]:
from PIL import Image

bad = []
for file in df['new_name']:
    try:
        Image.open(os.path.join(OUT_DIR, file)).verify()
    except:
        bad.append(file)

bad[:10], len(bad)


([], 0)

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI(api_key=api_key)


In [ ]:
models = client.models.list()
for m in models.data:
    print(m.id)


gpt-4-0613
gpt-4
gpt-3.5-turbo
chatgpt-image-latest
gpt-4o-mini-tts-2025-03-20
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2

In [ ]:
MODEL = "gpt-4o-mini"


In [ ]:
import random

FILES = sorted(glob(os.path.join(OUT_DIR, '*')))

# select 20 random images
random_sample = random.sample(FILES, 20)
FILES_20 = random_sample   # override the global FILES list

len(FILES_20), FILES_20[:5]


(20,
 ['/content/drive/MyDrive/hackathon_2025/defect_patches_png/01002.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00567.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00920.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00241.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00966.png'])

In [ ]:
# Colab-ready: batch classify via Responses API using inline base64 images
import os, base64, json, time, re
import pandas as pd
from glob import glob

RESULT_CSV = os.path.join(BASE_DIR, 'responses_results_gpt4o_mini.csv')

# FILES = sorted(glob(os.path.join(OUT_DIR, '*')))

# Small helper: extract first JSON-looking substring from text
def extract_json_from_text(text):
    """
    Extract the first {...} block that looks like JSON.
    Works even if the model outputs extra text.
    """
    # Find the first substring starting with '{' and ending with '}'
    # lazily, across multiple lines.
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, flags=re.DOTALL)

    for m in matches:
        try:
            return json.loads(m)
        except:
            pass

    return None

# ---------- classifier function (correct Responses image format) ----------
def classify_with_dataurl(path, model=MODEL, max_retries=3):
    with open(path, "rb") as f:
        raw = f.read()
    b64 = base64.b64encode(raw).decode("utf-8")
    data_url = f"data:image/png;base64,{b64}"

    core_instruction_0 = (
        "You are a materials-science assistant. Classify the image patch into one of: "
        "bulk, interstitial, Lomer-Cottrell dislocation, stacking fault, twin_bound, vacancy. "
        "Return strict JSON only: {\"c\":\"<class>\", \"e\":\"<1-2 sentence explanation>\", \"conf\":<0-1>}"
    )

    core_instruction_1 = (
        "You are a materials-science assistant analyzing 64×64 atomic-resolution CdTe patches. "
        "Classify the image into one of: bulk (defect-free), interstitial, Lomer–Cottrell dislocation, "
        "stacking fault, twin boundary, or vacancy. "
        "Definitions: bulk = ideal region with no defect; interstitial = extra atom at a non-lattice site; "
        "Lomer–Cottrell dislocation = sessile dislocation formed by two partials; stacking fault = deviation from "
        "the ideal stacking sequence; twin boundary = regions related by a crystallographic twin operation; "
        "vacancy = missing atom at a lattice site. "
        "Return strict JSON only: {\"c\":\"<class>\", \"e\":\"<1-2 sentence explanation>\", \"conf\":<0-1>}."
    )

    core_instruction_2 = (
        "You are a materials-science assistant analyzing 64×64 atomic-resolution CdTe patches. "
        "Classify the image into one of: bulk (defect-free), interstitial, Lomer–Cottrell dislocation, "
        "stacking fault, twin boundary, or vacancy. "
        "Visual cues: bulk = fully periodic dumbbell lattice with no anomalies; "
        "interstitial = an extra bright spot or dumbbell between normal lattice sites; "
        "Lomer–Cottrell dislocation = strong localized distortion forming a junction of bending lattice rows; "
        "stacking fault = a straight boundary where lattice rows shift phase across the plane; "
        "twin boundary = a straight boundary where the lattice shows mirror symmetry across the plane; "
        "vacancy = a dark site where a bright dumbbell should be. "
        "Return strict JSON only: {\"c\":\"<class>\", \"e\":\"<1-2 sentence explanation>\", \"conf\":<0-1>}."
    )

    core_instruction_combined = (
        "You are a materials-science assistant analyzing 64×64 atomic-resolution CdTe patches. "
        "Classify the image into one of: bulk (defect-free), interstitial, Lomer–Cottrell dislocation, "
        "stacking fault, twin boundary, or vacancy. "
        "Definitions (conceptual): bulk = an ideal defect-free region; interstitial = an extra atom at a non-lattice site; "
        "Lomer–Cottrell dislocation = a sessile node formed by two partial dislocations; stacking fault = deviation from the ideal stacking sequence; "
        "twin boundary = two regions related by a crystallographic twin operation; vacancy = a missing atom at a lattice site. "
        "Visual cues: bulk = fully periodic dumbbell pattern; interstitial = an extra bright spot between lattice sites; "
        "LC = a localized junction with bending/shifted lattice rows; SF = a straight boundary with row phase-shift; "
        "twin = a straight boundary showing mirror-like relation across it; vacancy = a dark/empty site where a dumbbell should be. "
        "Return strict JSON only: {\"c\":\"<class>\", \"e\":\"<1-2 sentence explanation>\", \"conf\":<0-1>}."
    )


    input_item = [{
        "role": "user",
        "content": [
            {"type": "input_text", "text": core_instruction_combined},
            {"type": "input_image", "image_url": data_url}
        ]
    }]

    backoff = 1.0
    for attempt in range(1, max_retries+1):
        try:
            resp = client.responses.create(model=model, input=input_item, max_output_tokens=250)
            raw_text = None
            if hasattr(resp, "output_text") and resp.output_text:
                raw_text = resp.output_text
            else:
                out = getattr(resp, "output", None) or resp.get("output", None)
                if out and len(out) > 0:
                    first = out[0].get("content", out[0].content if hasattr(out[0], "content") else None)
                    if isinstance(first, list):
                        for it in first:
                            if isinstance(it, dict) and it.get("text"):
                                raw_text = it.get("text"); break
                            elif hasattr(it, "text") and it.text:
                                raw_text = it.text; break
                    elif isinstance(first, dict):
                        raw_text = first.get("text") or getattr(first, "text", None)
            if raw_text is None:
                raw_text = str(resp)

            js = extract_json_from_text(raw_text)
            if js is None:
                return {"raw": raw_text, "ok": False}

            c = js.get("c") or js.get("label") or js.get("class")
            e = js.get("e") or js.get("explanation") or js.get("reason")
            conf = js.get("conf")
            try:
                conf = float(conf) if conf is not None else None
            except:
                conf = None

            return {"c": c, "e": e, "conf": conf, "raw": raw_text, "ok": True}

        except Exception as err:
            print(f"Attempt {attempt} failed: {err}; backing off {backoff}s")
            time.sleep(backoff)
            backoff *= 2.0

    return {"raw": None, "ok": False, "error": "max_retries_exceeded"}

# --- Batch loop with incremental saving ---
results = []
start_index = 0   # change if resuming from a partial run
for i, path in enumerate(FILES_20[start_index:], start=start_index):
    fname = os.path.basename(path)
    print(f"[{i+1}/{len(FILES_20)}] {fname}", flush=True)
    out = classify_with_dataurl(path)
    out_record = {
        "filename": fname,
        "model": MODEL,
        "pred_class": out.get("c"),
        "explanation": out.get("e"),
        "confidence": out.get("conf"),
        "ok": out.get("ok"),
        "raw": out.get("raw"),
        "error": out.get("error", "")
    }
    results.append(out_record)

    # save each N items to avoid data loss
    if (i+1) % 20 == 0 or (i+1) == len(FILES_20):
        pd.DataFrame(results).to_csv(RESULT_CSV, index=False)
        print(f"Saved {len(results)} results to {RESULT_CSV}")

[1/20] 01002.png
[2/20] 00567.png
[3/20] 00920.png
[4/20] 00241.png
[5/20] 00966.png
[6/20] 00973.png
[7/20] 00429.png
[8/20] 01069.png
[9/20] 01033.png
[10/20] 00718.png
[11/20] 00045.png
[12/20] 00692.png
[13/20] 00549.png
[14/20] 00947.png
[15/20] 00398.png
[16/20] 00209.png
[17/20] 00484.png
[18/20] 00979.png
[19/20] 00301.png
[20/20] 00079.png
Saved 20 results to /content/drive/MyDrive/hackathon_2025/responses_results_gpt4o_mini.csv


In [ ]:
import os, re
import pandas as pd

# which test CSV you generated
TEST_CSV = "/content/drive/MyDrive/hackathon_2025/responses_results_gpt4o_mini.csv"
MANIFEST_CSV = "/content/drive/MyDrive/hackathon_2025/manifest_png.csv"
OUT_AUG = os.path.join(BASE_DIR, "responses_results_gpt4o_mini.csv")
# canonical labels we expect
CANONICAL = ["bulk", "interstitial", "LC", "SF", "twin_bound", "vacancy"]
# ---------- end config ----------

# load files
results = pd.read_csv(TEST_CSV)
manifest = pd.read_csv(MANIFEST_CSV)

merged = results.merge(manifest, left_on="filename", right_on="new_name", how="left")

# function to map an original filename to one of canonical labels (or None)
def infer_label_from_name(name):
    if pd.isna(name):
        return None
    s = name.lower()
    # checks in order of specificity
    if "lomer" in s or "l-c" in s or re.search(r'\blc\b', s):
        return "LC"
    if "stack" in s or "stacking" in s or re.search(r'\bsf\b', s):
        return "SF"
    if "twin" in s:
        # map "twin bound" or "twin" -> twin_bound
        return "twin_bound"
    if "interst" in s or "interstitial" in s:
        return "interstitial"
    if "vac" in s or "vacancy" in s:
        return "vacancy"
    if "bulk" in s:
        return "bulk"
    # fallback: try to find any canonical substring
    for lab in CANONICAL:
        if lab.lower() in s:
            return lab
    return None

# normalize model prediction into same canonical label set
def normalize_pred(pred):
    if pd.isna(pred):
        return None
    p = str(pred).lower().strip()
    # common variants mapping
    if p in ["bulk", "bulk defect", "bulk_defect"]:
        return "bulk"
    if "interst" in p or "interstitial" in p:
        return "interstitial"
    if p in ["lc", "l c", "lomer-cottrell", "lomer cottrell", "lomer"]:
        return "LC"
    if p in ["sf", "stacking fault", "stacking_fault", "stacking"]:
        return "SF"
    if "twin" in p:
        return "twin_bound"
    if "vac" in p or "vacancy" in p:
        return "vacancy"
    # last-ditch: check substrings
    for lab in CANONICAL:
        if lab.lower() in p:
            return lab
    return p  # unknown label left as-is

# apply inference + normalization
merged["ground_truth"] = merged["original_name"].apply(infer_label_from_name)
merged["pred_norm"] = merged["pred_class"].apply(normalize_pred)

# compare (True/False/NaN)
merged["match"] = merged.apply(
    lambda r: (r["pred_norm"] == r["ground_truth"]) if (pd.notna(r["pred_norm"]) and pd.notna(r["ground_truth"])) else False,
    axis=1
)

# summary stats
total = len(merged)
correct = merged["match"].sum()
acc = correct / total if total>0 else 0.0

print(f"Total samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Accuracy: {acc:.3%}")

# per-class breakdown
print("\nPer-class (ground truth) accuracy:")
print(merged.groupby("ground_truth")["match"].agg(["count","sum","mean"]).rename(columns={"sum":"correct","mean":"accuracy"}))

# show mismatches for inspection
mismatches = merged[~merged["match"]].sort_values(by=["ground_truth","pred_norm"])
print("\nSample mismatches:")
display_cols = ["filename","original_name","ground_truth","pred_class","pred_norm","explanation","raw"]
print(mismatches[display_cols].head(20).to_string(index=False))

# save augmented CSV
merged.to_csv(OUT_AUG, index=False)
print(f"\nSaved augmented results to: {OUT_AUG}")


Total samples: 20
Correct predictions: 4
Accuracy: 20.000%

Per-class (ground truth) accuracy:
              count  correct  accuracy
ground_truth                          
LC                3        0  0.000000
SF                5        1  0.200000
bulk              2        2  1.000000
interstitial      6        1  0.166667
twin_bound        2        0  0.000000
vacancy           2        0  0.000000

Sample mismatches:
 filename        original_name ground_truth     pred_class pred_norm                                                                                                                                                       explanation                                                                                                                                                                                                         raw
00209.png            LC_12.png           LC stacking fault        SF                  The image shows a straight boundary with a clear pha

In [ ]:
import random

FILES = sorted(glob(os.path.join(OUT_DIR, '*')))

# select 200 random images
random_sample = random.sample(FILES, 200)
FILES_200 = random_sample   # override the global FILES list

len(FILES_200), FILES_200[:5]


(200,
 ['/content/drive/MyDrive/hackathon_2025/defect_patches_png/00066.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00542.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00737.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00211.png',
  '/content/drive/MyDrive/hackathon_2025/defect_patches_png/00665.png'])

In [ ]:
import os, base64, json, time, random
from glob import glob
import pandas as pd

# --- 1) Choose example files (provide paths to *your* chosen images) ---
# You should replace these picks with the clearest example files from your dataset.
# Option A: specify exact filenames (recommended)
prompt_examples = [
    {"image_path": os.path.join(OUT_DIR, "00000.png"), "text": "class: bulk. Reason: pristine, defect-free lattice with fully periodic dumbbell pattern."},
    {"image_path": os.path.join(OUT_DIR, "00964.png"), "text": "class: interstitial. Reason: extra bright atom/dumbbell located between regular lattice sites."},
    {"image_path": os.path.join(OUT_DIR, "00185.png"), "text": "class: LC. Reason: localized node where lattice rows bend at a junction from intersecting dislocations."},
    {"image_path": os.path.join(OUT_DIR, "00368.png"), "text": "class: SF. Reason: straight planar offset where lattice rows show a phase shift across a boundary."},
    {"image_path": os.path.join(OUT_DIR, "00739.png"), "text": "class: twin_bound. Reason: regions related by a twin operation (crystallographic twin relationship)."},
    {"image_path": os.path.join(OUT_DIR, "00913.png"), "text": "class: vacancy. Reason: a dark/missing dumbbell at a lattice site while surrounding lattice is intact."}
]
# --- NOTE: replace the filenames above with the actual files you want to use as examples ---

# --- helper: convert file to data URL ---
def file_to_data_url(p):
    with open(p,"rb") as f:
        b = f.read()
    b64 = base64.b64encode(b).decode("utf-8")
    return f"data:image/png;base64,{b64}"

# --- build prompt_examples_data with data URLs (for Requests) ---
prompt_examples_data = []
for ex in prompt_examples:
    assert os.path.exists(ex["image_path"]), f"Example not found: {ex['image_path']}"
    prompt_examples_data.append({
        "text": ex["text"],
        "data_url": file_to_data_url(ex["image_path"])
    })

# --- classifier that accepts prompt_examples_data (injects examples before main request) ---
def classify_with_fewshot_dataurl(target_path, prompt_examples_data, model=MODEL, max_retries=3):
    # build input list: examples first, then the target
    input_items = []
    # add each example as its own user content block
    for ex in prompt_examples_data:
        input_items.append({
            "role":"user",
            "content":[
                {"type":"input_text","text": ex["text"]},
                {"type":"input_image","image_url": ex["data_url"]}
            ]
        })

    # main instruction + target image
    core_instruction = (
        "You are a materials-science assistant analyzing 64×64 atomic-resolution CdTe patches. "
        "Classify the image into one of: bulk, interstitial, LC, SF, twin_bound, vacancy. "
        "Return strict JSON only: {\"c\":\"<class>\", \"e\":\"<1-2 sentence explanation>\", \"conf\":<0-1>}"
    )
    # add target
    data_url = file_to_data_url(target_path)
    input_items.append({
        "role":"user",
        "content":[
            {"type":"input_text","text": core_instruction},
            {"type":"input_image","image_url": data_url}
        ]
    })

    # call API with retries
    backoff = 1.0
    for attempt in range(1, max_retries+1):
        try:
            resp = client.responses.create(model=model, input=input_items, max_output_tokens=200, temperature=0.0)
            # robustly extract text (same approach you used before)
            raw_text = None
            if hasattr(resp, "output_text") and resp.output_text:
                raw_text = resp.output_text
            else:
                out = getattr(resp, "output", None) or resp.get("output", None)
                if out and len(out)>0:
                    first = out[0].get("content", out[0].content if hasattr(out[0],"content") else None)
                    if isinstance(first, list):
                        for it in first:
                            if isinstance(it, dict) and it.get("text"):
                                raw_text = it.get("text"); break
                            elif hasattr(it, "text") and it.text:
                                raw_text = it.text; break
                    elif isinstance(first, dict):
                        raw_text = first.get("text") or getattr(first,"text",None)
            if raw_text is None:
                raw_text = str(resp)

            # extract JSON (simple extractor)
            import re, json
            m = re.search(r'\{.*?\}', raw_text, flags=re.DOTALL)
            if not m:
                return {"raw": raw_text, "ok": False}
            js = json.loads(m.group(0))
            c = js.get("c") or js.get("label") or js.get("class")
            e = js.get("e") or js.get("explanation") or js.get("reason")
            conf = js.get("conf")
            try: conf = float(conf) if conf is not None else None
            except: conf = None
            return {"c": c, "e": e, "conf": conf, "raw": raw_text, "ok": True}
        except Exception as err:
            print(f"Attempt {attempt} failed: {err}; backing off {backoff}s")
            time.sleep(backoff)
            backoff *= 2.0
    return {"raw": None, "ok": False, "error": "max_retries_exceeded"}

# --- Run test on 20 random images (not using any of the example files) ---
# all_files = sorted(glob(os.path.join(OUT_DIR,"*.png")))
# # exclude prompt examples from test set
# example_paths = set([os.path.abspath(e["image_path"]) for e in prompt_examples])
# test_candidates = [p for p in all_files if os.path.abspath(p) not in example_paths]

# sample = random.sample(test_candidates, min(TEST_SIZE, len(test_candidates)))
sample = FILES_200

test_results = []
for i, p in enumerate(sample, start=1):
    print(f"[{i}/{len(sample)}] {os.path.basename(p)}")
    r = classify_with_fewshot_dataurl(p, prompt_examples_data)
    test_results.append({
        "filename": os.path.basename(p),
        "pred_class": r.get("c"),
        "explanation": r.get("e"),
        "confidence": r.get("conf"),
        "ok": r.get("ok"),
        "raw": r.get("raw")
    })

out_file = os.path.join(BASE_DIR, "test_results_fewshot_gpt4o_mini.csv")
pd.DataFrame(test_results).to_csv(out_file, index=False)
print("Saved few-shot test results to", out_file)


[1/200] 00066.png
[2/200] 00542.png
[3/200] 00737.png
[4/200] 00211.png
[5/200] 00665.png
[6/200] 00474.png
[7/200] 00348.png
[8/200] 00593.png
[9/200] 00319.png
[10/200] 00365.png
[11/200] 00069.png
[12/200] 00033.png
[13/200] 00465.png
[14/200] 00726.png
[15/200] 00782.png
[16/200] 00352.png
[17/200] 00580.png
[18/200] 00995.png
[19/200] 00313.png
[20/200] 00485.png
[21/200] 00953.png
[22/200] 00077.png
[23/200] 00283.png
[24/200] 01102.png
[25/200] 00851.png
[26/200] 00913.png
[27/200] 00164.png
[28/200] 00865.png
[29/200] 00711.png
[30/200] 00366.png
[31/200] 00994.png
[32/200] 00553.png
[33/200] 00228.png
[34/200] 01062.png
[35/200] 00289.png
[36/200] 00000.png
[37/200] 00096.png
[38/200] 00864.png
[39/200] 00832.png
[40/200] 00841.png
[41/200] 01084.png
[42/200] 00507.png
[43/200] 00520.png
[44/200] 00654.png
[45/200] 00326.png
[46/200] 00617.png
[47/200] 00657.png
[48/200] 01061.png
[49/200] 00689.png
[50/200] 00783.png
[51/200] 00640.png
[52/200] 00705.png
[53/200] 00717.png
[5

In [ ]:
import os, re
import pandas as pd

# which test CSV you generated
TEST_CSV = "/content/drive/MyDrive/hackathon_2025/test_results_fewshot_gpt4o_mini.csv"
MANIFEST_CSV = "/content/drive/MyDrive/hackathon_2025/manifest_png.csv"
OUT_AUG = os.path.join(BASE_DIR, "test_results_fewshot_gpt4o_mini.csv")
# canonical labels we expect
CANONICAL = ["bulk", "interstitial", "LC", "SF", "twin_bound", "vacancy"]
# ---------- end config ----------

# load files
results = pd.read_csv(TEST_CSV)
manifest = pd.read_csv(MANIFEST_CSV)

merged = results.merge(manifest, left_on="filename", right_on="new_name", how="left")

# function to map an original filename to one of canonical labels (or None)
def infer_label_from_name(name):
    if pd.isna(name):
        return None
    s = name.lower()
    # checks in order of specificity
    if "lomer" in s or "l-c" in s or re.search(r'\blc\b', s):
        return "LC"
    if "stack" in s or "stacking" in s or re.search(r'\bsf\b', s):
        return "SF"
    if "twin" in s:
        # map "twin bound" or "twin" -> twin_bound
        return "twin_bound"
    if "interst" in s or "interstitial" in s:
        return "interstitial"
    if "vac" in s or "vacancy" in s:
        return "vacancy"
    if "bulk" in s:
        return "bulk"
    # fallback: try to find any canonical substring
    for lab in CANONICAL:
        if lab.lower() in s:
            return lab
    return None

# normalize model prediction into same canonical label set
def normalize_pred(pred):
    if pd.isna(pred):
        return None
    p = str(pred).lower().strip()
    # common variants mapping
    if p in ["bulk", "bulk defect", "bulk_defect"]:
        return "bulk"
    if "interst" in p or "interstitial" in p:
        return "interstitial"
    if p in ["lc", "l c", "lomer-cottrell", "lomer cottrell", "lomer"]:
        return "LC"
    if p in ["sf", "stacking fault", "stacking_fault", "stacking"]:
        return "SF"
    if "twin" in p:
        return "twin_bound"
    if "vac" in p or "vacancy" in p:
        return "vacancy"
    # last-ditch: check substrings
    for lab in CANONICAL:
        if lab.lower() in p:
            return lab
    return p  # unknown label left as-is

# apply inference + normalization
merged["ground_truth"] = merged["original_name"].apply(infer_label_from_name)
merged["pred_norm"] = merged["pred_class"].apply(normalize_pred)

# compare (True/False/NaN)
merged["match"] = merged.apply(
    lambda r: (r["pred_norm"] == r["ground_truth"]) if (pd.notna(r["pred_norm"]) and pd.notna(r["ground_truth"])) else False,
    axis=1
)

# summary stats
total = len(merged)
correct = merged["match"].sum()
acc = correct / total if total>0 else 0.0

print(f"Total samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Accuracy: {acc:.3%}")

# per-class breakdown
print("\nPer-class (ground truth) accuracy:")
print(merged.groupby("ground_truth")["match"].agg(["count","sum","mean"]).rename(columns={"sum":"correct","mean":"accuracy"}))

# show mismatches for inspection
mismatches = merged[~merged["match"]].sort_values(by=["ground_truth","pred_norm"])
print("\nSample mismatches:")
display_cols = ["filename","original_name","ground_truth","pred_class","pred_norm","explanation","raw"]
print(mismatches[display_cols].head(20).to_string(index=False))

# save augmented CSV
merged.to_csv(OUT_AUG, index=False)
print(f"\nSaved augmented results to: {OUT_AUG}")


Total samples: 200
Correct predictions: 102
Accuracy: 51.000%

Per-class (ground truth) accuracy:
              count  correct  accuracy
ground_truth                          
LC               34       31  0.911765
SF               55       14  0.254545
bulk             28       28  1.000000
interstitial     22       14  0.636364
twin_bound       27        6  0.222222
vacancy          34        9  0.264706

Sample mismatches:
 filename original_name ground_truth pred_class  pred_norm                                                                                                                                                                                                   explanation                                                                                                                                                                                                                                                           raw
00264.png     LC_17.png           LC twin_bound twi